In [ ]:
from ase.build import bulk
import io
from ase.visualize import view
from ase.io import read, write
import numpy as np
import os
import fileinput
from shutil import rmtree
from shutil import copyfile
from ase.phonons import Phonons
import matplotlib.pyplot as plt
from ase.build import make_supercell
from ase.optimize import LBFGS
from acease.ace_calculator import ACEpotentials, initialize_julia, set_params
from ase.constraints import UnitCellFilter
from ase.units import J
from ase.constraints import FixCartesian
import pandas as pd

In [ ]:
model_path = "/home/f/PhD/ace_pot/model.json"
python_path = "/home/f/PhD/env/bin/python"
file_path_0 = '/home/f/PhD/ace_pot/main_model_just_connor.csv'
file_path_1 = '/home/f/PhD/ace_pot/20_percent_models_all_plannar_defects/final_model_no_stack_no_twin.csv'
file_path_2 = '/home/f/PhD/ace_pot/20_percent_models_all_plannar_defects/final_model_no_twin.csv'
file_path_3 = '/home/f/PhD/ace_pot/20_percent_models_all_plannar_defects/final_model_no_twin_active_learned.csv'
m1 = '/home/f/PhD/ace_pot/five_models_no_defects/coeffs_subset_1.csv'
m2 = '/home/f/PhD/ace_pot/five_models_no_defects/coeffs_subset_2.csv'
m3 = '/home/f/PhD/ace_pot/five_models_no_defects/coeffs_subset_3.csv'
m4 = '/home/f/PhD/ace_pot/five_models_no_defects/coeffs_subset_4.csv'
m5 = '/home/f/PhD/ace_pot/five_models_no_defects/coeffs_subset_5.csv'
g1 = '/home/f/PhD/ace_pot/five_models_stacking_faults/coeffs_subset_1.csv'
g2 = '/home/f/PhD/ace_pot/five_models_stacking_faults/coeffs_subset_2.csv'
g3 = '/home/f/PhD/ace_pot/five_models_stacking_faults/coeffs_subset_3.csv'
g4 = '/home/f/PhD/ace_pot/five_models_stacking_faults/coeffs_subset_4.csv'
g5 = '/home/f/PhD/ace_pot/five_models_stacking_faults/coeffs_subset_5.csv'

initialize_julia(python_path)  # pass your python path

In [ ]:
calculator = ACEpotentials(model_path) #This is specifically and ace calc

In [ ]:
struct = bulk('Ti', 'hcp', a=2.7,c=4.6)
struct.calc = calculator
print(struct.get_potential_energy())
#if there is an output, mean model is working!!!

In [ ]:
''' Hello, this is for stacking faults: the set up has been complete and the following is going to be done:

    We are going to analyse a few different stacking faults: Namely: Basal, Pris 1, Pris 2, Pyrm 1, Pyrm 2

    1. Create Gamma surface for each trained on basal stacking:
    2. sample 10 points using LHC sampling, and perturb the system each sample found
    3. Run DFT on these configs
    4. Use those to make better ACE model
    5. Look at the difference between just using the Connors data base and the stacking fault informed one
'''

In [ ]:
from scipy.stats import qmc
from scipy.interpolate import griddata
def create_lhc_mesh(dimensions=2, samples=20):
    """
    Create a Latin Hypercube Sampling (LHS) mesh.

    Parameters:
    -----------
    dimensions : int
        Number of dimensions (2 for a 10x10 grid).
    samples : int
        Number of sample points (10x10 = 100).

    Returns:
    --------
    np.ndarray
        Array of shape (samples, dimensions) containing LHS mesh points.
    """
    sampler = qmc.LatinHypercube(d=dimensions)
    lhc = sampler.random(n=samples)
    
    # Scale to [0,10] for a 10x10 grid
    mesh = qmc.scale(lhc, [0, 0], [1, 1])
    return mesh

def I2_mesh_shear(bulk,v1,v2):

    # Create a mesh of stacking fault configurations (unrelaxed)
    v1 = np.array(v1).flatten()
    v2 = np.array(v2).flatten()
    mesh_width = 10
    original_cell = bulk.cell.copy()
    # Clear or initialize the output file
    # output_file='path.xyz'
    # output_dir = 'gamma_surface'
    # os.makedirs(output_dir, exist_ok=True)
    # Clear or initialize the output file
    # open(output_file, 'w').close()
    count = 0
    original_cell = bulk.cell.copy()
    energy_grid = np.zeros((mesh_width, mesh_width))
    images = []
    for i in range(mesh_width):
        for j in range(mesh_width):
            shift = (i / (mesh_width - 1)) * v1 + (j / (mesh_width - 1)) * v2
            
            bulk_copy = bulk.copy()
            bulk_copy.set_cell(original_cell)
            bulk_copy.set_cell([bulk_copy.cell[0],bulk_copy.cell[1],bulk_copy.cell[2]+shift])
            bulk_copy.wrap()
            bulk_copy.calc = calculator

            constraints = [FixCartesian(i, [True, True, False]) for i in range(len(bulk_copy))]
            bulk_copy.set_constraint(constraints)
            mask = [[False, False, False],
                [False, False, False],
                [False, False, True]]
            ucf = UnitCellFilter(bulk_copy, mask= mask)

            dyn = LBFGS(ucf)
            dyn.run(fmax=0.01)
            # write(output_file, bulk_copy, append=True)
            e = bulk_copy.get_potential_energy()
            if(i == 0 and j == 0):
                bulk_energy = e
            surface_size = np.linalg.norm(np.cross(bulk_copy.cell[0],bulk_copy.cell[1]))
            energy_grid[i, j] = (e-bulk_energy)*(1000)/(J*((1e-10)**2)*surface_size)
            images.append(bulk_copy)
            # cell_file = os.path.join(output_dir, f"{count}.cell")
            
            count += 1
            # write(cell_file, bulk_copy, format='castep-cell')


            bulk_copy.calc = None
    write("path.xyz",images)
    print(v1)
    print(v2)
    x = np.linspace(0, 1, mesh_width)
    y = np.linspace(0, 1, mesh_width)
    X, Y = np.meshgrid(x, y)

    plt.figure(figsize=(6, 5))
    cp = plt.contourf(X, Y, energy_grid.T, levels=50, cmap='viridis')
    plt.colorbar(cp, label='Energy (mJ/m^2)')
    plt.xlabel('Shift along v1 (fractional)')
    plt.ylabel('Shift along v2 (fractional)')
    plt.title('Stacking Fault Energy Surface')
    plt.tight_layout()
    plt.savefig("energy_surface_contour.png", dpi=300)
    plt.show()

def change_calc(struct,calculator,parameters):
    struct_copy = struct.copy()
    struct = bulk('Ti', 'hcp', a=2.7,c=10)
    calculator = set_params(calculator, parameters)
    struct.calc = calculator
    struct.get_potential_energy()
    struct = struct_copy.copy()
    return calculator

def lhc_gamma_surface_samples(bulk,v1,v2,file_path):
    #This works for generating samples well! Very happy with this!
    energies = []
    original_cell = bulk.cell.copy()
    samples = create_lhc_mesh()
    images = []
    for i in range(len(samples)):
        shift = samples[i,0] * v1 + samples[i,1]  * v2
        
        bulk_copy = bulk.copy()
        bulk_copy.set_cell(original_cell)
        bulk_copy.set_cell([bulk_copy.cell[0],bulk_copy.cell[1],bulk_copy.cell[2]+shift])
        bulk_copy.wrap()
        bulk_copy.calc = calculator
        constraints = [FixCartesian(i, [True, True, False]) for i in range(len(bulk_copy))]
        bulk_copy.set_constraint(constraints)
        mask = [[False, False, False],
            [False, False, False],
            [False, False, True]]
        ucf = UnitCellFilter(bulk_copy, mask= mask)

        dyn = LBFGS(ucf)
        dyn.run(fmax=0.001)
        # write(output_file, bulk_copy, append=True)
        e = bulk_copy.get_potential_energy()
        if(i == 0):
            bulk_energy = e
        surface_size = np.linalg.norm(np.cross(bulk_copy.cell[0],bulk_copy.cell[1]))
        energies.append((e-bulk_energy)*(1000)/(J*((1e-10)**2)*surface_size))
        images.append(bulk_copy)
        


        bulk_copy.calc = None
    write(file_path,images)
    grid_x, grid_y = np.mgrid[0:1:200j, 0:1:200j]  # 200x200 grid

    # interpolate scattered data onto grid
    energy_grid = griddata(samples, energies, (grid_x, grid_y), method='cubic')

    # plot contour
    plt.figure(figsize=(6,5))
    cp = plt.contourf(grid_x, grid_y, energy_grid, levels=50, cmap='viridis')
    plt.colorbar(cp, label='Energy (mJ/m^2)')
    plt.xlabel('Shift along v1 (fractional)')
    plt.ylabel('Shift along v2 (fractional)')
    plt.title('Stacking Fault Energy Surface')
    plt.tight_layout()
    plt.savefig("energy_surface_contour.png", dpi=300)
    plt.show()


In [ ]:


supercell_basal = read("/home/f/PhD/ace_pot/supercells/0_0_0_1.cfg",format='cfg')
file_path =  file_path_1
df = pd.read_csv(file_path,header = None)
parameters = np.array(df)
params = parameters.ravel()

calculator = change_calc(supercell_basal,calculator,params)
supercell_basal = supercell_basal *(1,1,10)
v1 = supercell_basal.cell[0]
v2 = supercell_basal.cell[1]
file_path = "/home/f/PhD/ace_pot/lhc_gamma_surface_samples/basal.xyz"
lhc_gamma_surface_samples(supercell_basal,v1,v2,file_path)

In [ ]:
supercell_prism_1 = read("/home/f/PhD/ace_pot/supercells/prism1_correct.xyz")
for i in range(len(supercell_prism_1)):
    supercell_prism_1.symbols[i] = 'Ti'
supercell_prism_1 = supercell_prism_1 *(1,1,10)
v1 = supercell_prism_1.cell[0]
v2 = supercell_prism_1.cell[1]
file_path = "/home/f/PhD/ace_pot/lhc_gamma_surface_samples/prism1_W.xyz"
lhc_gamma_surface_samples(supercell_prism_1,v1,v2,file_path)

In [ ]:
supercell_prism_2 = read("/home/f/PhD/ace_pot/supercells/_-2_1_1_0.cfg",format='cfg')
supercell_prism_2 = supercell_prism_2 * (1,1,10)
v1 = supercell_prism_2.cell[0]
v2 = supercell_prism_2.cell[1]
file_path = "/home/f/PhD/ace_pot/lhc_gamma_surface_samples/prism2.xyz"
lhc_gamma_surface_samples(supercell_prism_2,v1,v2,file_path)

In [ ]:
supercell_pyrm_1 = read("/home/f/PhD/ace_pot/supercells/1_0_-1_1.cfg",format='cfg')
supercell_pyrm_1 = supercell_pyrm_1 * (1,1,2)
v1 = supercell_pyrm_1.cell[0]
v2 = supercell_pyrm_1.cell[1]
file_path = "/home/f/PhD/ace_pot/lhc_gamma_surface_samples/pyrm1_W.xyz"
lhc_gamma_surface_samples(supercell_pyrm_1,v1,v2,file_path)

In [ ]:
supercell_pyrm_2 = read("/home/f/PhD/ace_pot/supercells/1_1_-2_2.cfg",format='cfg')
supercell_pyrm_2 = supercell_pyrm_2 * (1,1,2)
v1 = supercell_pyrm_2.cell[0]
v2 = supercell_pyrm_2.cell[1]
file_path = "/home/f/PhD/ace_pot/lhc_gamma_surface_samples/pyrm2.xyz"
lhc_gamma_surface_samples(supercell_pyrm_2,v1,v2,file_path)

In [ ]:
#Now we look at the gamma line that we can compare with the paper on the stacking faults

# first we start with the with I2 stacking fault for proof of concept

def create_gamma_line(bulk,slip_direction,file_name,calculator,pointer):
    plt.figure(figsize=(6, 6))
    # Create a mesh of stacking fault configurations (unrelaxed)
    number_of_points = 11
    x = np.linspace(0, 0.6, number_of_points)
    original_cell = bulk.cell.copy()
    original_cell = bulk.cell.copy()
    energies = []
    images = []
    all_energies = []
    all_energies_gamma = []
    for j in range(5):
        energies = []
        images = []
        if j == 0:
            file_path = m1
            df = pd.read_csv(file_path,header = None)
            parameters = np.array(df)
            params = parameters.ravel()
            calculator = change_calc(bulk,calculator,params)
        elif j == 1:
            file_path = m2
            df = pd.read_csv(file_path,header = None)
            parameters = np.array(df)
            params = parameters.ravel()
            calculator = change_calc(bulk,calculator,params)
        elif j == 2:
            file_path = m3
            df = pd.read_csv(file_path,header = None)
            parameters = np.array(df)
            params = parameters.ravel()
            calculator = change_calc(bulk,calculator,params)
        elif j == 3:
            file_path = m4
            df = pd.read_csv(file_path,header = None)
            parameters = np.array(df)
            params = parameters.ravel()
            calculator = change_calc(bulk,calculator,params)
        elif j == 4:
            file_path = m5
            df = pd.read_csv(file_path,header = None)
            parameters = np.array(df)
            params = parameters.ravel()
            calculator = change_calc(bulk,calculator,params)
        for i in range(number_of_points):
            shift = (slip_direction * (1/(number_of_points-1))*i)*0.6
            
            bulk_copy = bulk.copy()
            bulk_copy.set_cell(original_cell)
            bulk_copy.set_cell([bulk_copy.cell[0],bulk_copy.cell[1],bulk_copy.cell[2]+shift])
            bulk_copy.wrap()
            bulk_copy.calc = calculator

            constraints = [FixCartesian(con, [True, True, False]) for con in range(len(bulk_copy))]
            bulk_copy.set_constraint(constraints)
            mask = [[False, False, False],
                [False, False, False],
                [False, False, True]]
            ucf = UnitCellFilter(bulk_copy, mask= mask)

            dyn = LBFGS(ucf)
            dyn.run(fmax=0.0001)
            # write(output_file, bulk_copy, append=True)
            
            #comment if not needing the castep

            e = bulk_copy.get_potential_energy()
            if(i == 0):
                bulk_energy = e
            surface_size = np.linalg.norm(np.cross(bulk_copy.cell[0],bulk_copy.cell[1]))
            energies.append((e-bulk_energy)*(1000)/(J*((1e-10)**2)*surface_size))
            images.append(bulk_copy)
            # cell_file = os.path.join(output_dir, f"{count}.cell")
            bulk_copy.calc = None
            all_energies.append(energies)
    


    for j in range(5):
        energies = []
        images = []
        if j == 0:
            file_path = g1
            df = pd.read_csv(file_path,header = None)
            parameters = np.array(df)
            params = parameters.ravel()
            calculator = change_calc(bulk,calculator,params)
        elif j == 1:
            file_path = g2
            df = pd.read_csv(file_path,header = None)
            parameters = np.array(df)
            params = parameters.ravel()
            calculator = change_calc(bulk,calculator,params)
        elif j == 2:
            file_path = g3
            df = pd.read_csv(file_path,header = None)
            parameters = np.array(df)
            params = parameters.ravel()
            calculator = change_calc(bulk,calculator,params)
        elif j == 3:
            file_path = g4
            df = pd.read_csv(file_path,header = None)
            parameters = np.array(df)
            params = parameters.ravel()
            calculator = change_calc(bulk,calculator,params)
        elif j == 4:
            file_path = g5
            df = pd.read_csv(file_path,header = None)
            parameters = np.array(df)
            params = parameters.ravel()
            calculator = change_calc(bulk,calculator,params)
        for i in range(number_of_points):
            shift = (slip_direction * (1/(number_of_points-1))*i)*0.6
            
            bulk_copy = bulk.copy()
            bulk_copy.set_cell(original_cell)
            bulk_copy.set_cell([bulk_copy.cell[0],bulk_copy.cell[1],bulk_copy.cell[2]+shift])
            bulk_copy.wrap()
            bulk_copy.calc = calculator

            constraints = [FixCartesian(con, [True, True, False]) for con in range(len(bulk_copy))]
            bulk_copy.set_constraint(constraints)
            mask = [[False, False, False],
                [False, False, False],
                [False, False, True]]
            ucf = UnitCellFilter(bulk_copy, mask= mask)

            dyn = LBFGS(ucf)
            dyn.run(fmax=0.0001)
            # write(output_file, bulk_copy, append=True)
            
            #comment if not needing the castep

            e = bulk_copy.get_potential_energy()
            if(i == 0):
                bulk_energy = e
            surface_size = np.linalg.norm(np.cross(bulk_copy.cell[0],bulk_copy.cell[1]))
            energies.append((e-bulk_energy)*(1000)/(J*((1e-10)**2)*surface_size))
            images.append(bulk_copy)
            # cell_file = os.path.join(output_dir, f"{count}.cell")
            bulk_copy.calc = None
            all_energies_gamma.append(energies)



    all_energies = np.array(all_energies) 
    mean_energies = np.mean(all_energies, axis=0)
    std_energies = np.std(all_energies, axis=0)
    plt.plot(x, mean_energies, label="no_defects Mean", color="blue")
    plt.fill_between(x,
                     mean_energies - std_energies,
                     mean_energies + std_energies,
                     alpha=0.3, color="blue", label="±1 std (no_defects)")
    
    all_energies_gamma = np.array(all_energies_gamma) 
    mean_energies = np.mean(all_energies_gamma, axis=0)
    std_energies = np.std(all_energies_gamma, axis=0)
    plt.plot(x, mean_energies, label="gamma_informed Mean", color="red")
    plt.fill_between(x,
                     mean_energies - std_energies,
                     mean_energies + std_energies,
                     alpha=0.3, color="red", label="±1 std (gamma_informed)")











    df = pd.read_csv("data_points_gamma_lines.csv", header=None, names=["x", "y"])

    # Extract only the y values (second column)
    y_values = df["y"].tolist()

    # Break into 4 groups of 11 consecutive points
    groups = [y_values[i:i+11] for i in range(0, 44, 11)]
    plt.plot(x,groups[pointer], '-o',label = "Literature (Vasp Relaxed)",color = "green")
    

    #Now we extra DFT along the gamma line that we have!
    images = read("my_and_connor.xyz", index=":")
    qm_energies = [atoms.info.get("QM_energy") for atoms in images]
    qm_energies_plot = []
    qm_energies_plot.append(0)
    for e in qm_energies[6970+pointer*10:6980+pointer*10]:
        qm_energies_plot.append((e-bulk_energy)*(1000)/(J*((1e-10)**2)*surface_size))
    plt.plot(x, qm_energies_plot, '-o', label="DFT (Relaxed with ACE)",color = "orange")


    basal_I2_point =  (-63753.43641938  - bulk_energy)*(1000)/(J*((1e-10)**2)*surface_size)
    prism1_point = (-63753.39751906- bulk_energy)*(1000)/(J*((1e-10)**2)*surface_size)
    pyrm1_point  = (-82879.43240445- bulk_energy)*(1000)/(J*((1e-10)**2)*surface_size)
    pyrm2_point =  (-89254.25363093- bulk_energy)*(1000)/(J*((1e-10)**2)*surface_size)
    if pointer == 1:
        plt.scatter(x[8], prism1_point, color="black", marker="D", 
                label="Relaxed Reference Point", zorder=10)
    elif pointer == 2:
        plt.scatter(x[8], pyrm1_point, color="black", marker="D", 
                    label="Relaxed Reference Point", zorder=10)
    elif pointer == 3:
        plt.scatter(x[7], pyrm2_point, color="black", marker="D", 
                    label="Relaxed Reference Point", zorder=10)



    plt.tight_layout()
    plt.ylabel(r'mJ/m$^2$', fontsize=18)
    plt.xticks(fontsize=18)
    plt.yticks(fontsize=18)
    plt.grid(True, ls="--", alpha=0.6)
    if pointer == 0 or pointer == 1:
        plt.legend(fontsize = 15)
    plt.savefig("energy_surface_contour.png", dpi=300)
    plt.show()



In [ ]:
# supercell_basal = read("/home/f/PhD/ace_pot/supercells/0_0_0_1.cfg",format='cfg')
# #file_path = '/home/f/PhD/ace_pot/5_models/model_1.csv'
# file_path = file_path_1
# df = pd.read_csv(file_path,header = None)
# parameters = np.array(df)
# params = parameters.ravel()
# calculator = change_calc(supercell_basal,calculator,params)

# supercell_basal = supercell_basal *(1,1,10)
# slip_vec = np.array([-4.4085,2.54524866,0])

# create_gamma_line(supercell_basal,slip_vec)

# supercell_basal = read("/home/f/PhD/ace_pot/supercells/0_0_0_1.cfg",format='cfg')
# #file_path = '/home/f/PhD/ace_pot/5_models/model_1.csv'
# file_path = file_path_2
# df = pd.read_csv(file_path,header = None)
# parameters = np.array(df)
# params = parameters.ravel()
# calculator = change_calc(supercell_basal,calculator,params)

# supercell_basal = supercell_basal *(1,1,10)
# slip_vec = np.array([-4.4085,2.54524866,0])

# create_gamma_line(supercell_basal,slip_vec,"gamma_line_basal.xyz")

supercell_basal = read("/home/f/PhD/ace_pot/supercells/0_0_0_1.cfg",format='cfg')
#file_path = '/home/f/PhD/ace_pot/5_models/model_1.csv'
file_path = file_path_3
df = pd.read_csv(file_path,header = None)
parameters = np.array(df)
params = parameters.ravel()
calculator = change_calc(supercell_basal,calculator,params)

supercell_basal = supercell_basal *(1,1,20)
slip_vec = np.array([-4.4085,2.54524866,0])

create_gamma_line(supercell_basal,slip_vec,"gamma_line_basal.xyz",calculator,0)


In [ ]:
# # prism 1 gamma
# file_path = file_path_1
# df = pd.read_csv(file_path,header = None)
# parameters = np.array(df)
# params = parameters.ravel()
# calculator = change_calc(supercell_basal,calculator,params)
# supercell_prism_1 = read("/home/f/PhD/ace_pot/supercells/prism1_correct.xyz")
# for i in range(len(supercell_prism_1)):
#     supercell_prism_1.symbols[i] = 'Ti'
# supercell_prism_1 = supercell_prism_1 *(1,1,10)
# slip_vec = np.array([0,2.939,0])
# create_gamma_line(supercell_prism_1,slip_vec)


# file_path = file_path_2
# df = pd.read_csv(file_path,header = None)
# parameters = np.array(df)
# params = parameters.ravel()
# calculator = change_calc(supercell_basal,calculator,params)
# supercell_prism_1 = read("/home/f/PhD/ace_pot/supercells/prism1_correct.xyz")
# for i in range(len(supercell_prism_1)):
#     supercell_prism_1.symbols[i] = 'Ti'
# supercell_prism_1 = supercell_prism_1 *(1,1,10)
# slip_vec = np.array([0,2.939,0])
# create_gamma_line(supercell_prism_1,slip_vec,"gamma_line_prism1.xyz")

file_path = file_path_3
df = pd.read_csv(file_path,header = None)
parameters = np.array(df)
params = parameters.ravel()
calculator = change_calc(supercell_basal,calculator,params)
supercell_prism_1 = read("/home/f/PhD/ace_pot/supercells/prism1_correct.xyz")
for i in range(len(supercell_prism_1)):
    supercell_prism_1.symbols[i] = 'Ti'
supercell_prism_1 = supercell_prism_1 *(1,1,10)
slip_vec = np.array([0,2.939,0])
create_gamma_line(supercell_prism_1,slip_vec,"gamma_line_prism1.xyz",calculator,1)

In [ ]:
# slip_vec = np.array([-5.10565500,-1.41604618,0])
slip_vec = np.array([0,5.29838652,0])
theta = np.deg2rad(0)

# 2D rotation matrix
R = np.array([
    [np.cos(theta), -np.sin(theta)],
    [np.sin(theta),  np.cos(theta)]
])

# apply rotation to x,y part
rotated_xy = R @ slip_vec[:2]

# put back into 3D (z stays the same)
slip_vec = np.array([rotated_xy[0], rotated_xy[1], slip_vec[2]])


# file_path = file_path_1
# df = pd.read_csv(file_path,header = None)
# parameters = np.array(df)
# params = parameters.ravel()
# calculator = change_calc(supercell_basal,calculator,params)
# supercell_pyrm_1 = read("/home/f/PhD/ace_pot/supercells/1_0_-1_1.cfg",format='cfg')
# supercell_pyrm_1 = supercell_pyrm_1 * (1,1,1)
# v1 = supercell_pyrm_1.cell[0]
# v2 = supercell_pyrm_1.cell[1]
# create_gamma_line(supercell_pyrm_1,-slip_vec,"one_toeasd.xyz")

# file_path = file_path_2
# df = pd.read_csv(file_path,header = None)
# parameters = np.array(df)
# params = parameters.ravel()
# calculator = change_calc(supercell_basal,calculator,params)
# supercell_pyrm_1 = read("/home/f/PhD/ace_pot/supercells/1_0_-1_1.cfg",format='cfg')
# supercell_pyrm_1 = supercell_pyrm_1 * (1,1,1)
# v1 = supercell_pyrm_1.cell[0]
# v2 = supercell_pyrm_1.cell[1]



# print(slip_vec)



# create_gamma_line(supercell_pyrm_1,-slip_vec,"gamma_line_pyrm1.xyz")

# file_path = file_path_3
# df = pd.read_csv(file_path,header = None)
# parameters = np.array(df)
# params = parameters.ravel()
# calculator = change_calc(supercell_basal,calculator,params)
supercell_pyrm_1 = read("/home/f/PhD/ace_pot/supercells/1_0_-1_1.cfg",format='cfg')
supercell_pyrm_1 = supercell_pyrm_1 * (1,1,2)
v1 = supercell_pyrm_1.cell[0]
v2 = supercell_pyrm_1.cell[1]
# slip_vec = np.array([-5.10565500,-1.41604618,0])
create_gamma_line(supercell_pyrm_1,slip_vec,"gamma_line_pyrm1.xyz",calculator,2)

In [ ]:
# file_path = file_path_1
# df = pd.read_csv(file_path,header = None)
# parameters = np.array(df)
# params = parameters.ravel()
# calculator = change_calc(supercell_basal,calculator,params)
# supercell_pyrm_2 = read("/home/f/PhD/ace_pot/supercells/1_1_-2_2.cfg",format='cfg')
# supercell_pyrm_2 = supercell_pyrm_2 * (1,1,1)
# v1 = supercell_pyrm_2.cell[0]
# v2 = supercell_pyrm_2.cell[1]
# slip_vec = np.array([0,5.49839340,0])
# create_gamma_line(supercell_pyrm_2,-slip_vec,"gamma_line_pyrm2.xyz")

# file_path = file_path_2
# df = pd.read_csv(file_path,header = None)
# parameters = np.array(df)
# params = parameters.ravel()
# calculator = change_calc(supercell_basal,calculator,params)
# supercell_pyrm_2 = read("/home/f/PhD/ace_pot/supercells/1_1_-2_2.cfg",format='cfg')
# supercell_pyrm_2 = supercell_pyrm_2 * (1,1,1)
# v1 = supercell_pyrm_2.cell[0]
# v2 = supercell_pyrm_2.cell[1]
# slip_vec = np.array([0,5.49839340,0])
# create_gamma_line(supercell_pyrm_2,-slip_vec,"gamma_line_pyrm2.xyz")

file_path = file_path_3
df = pd.read_csv(file_path,header = None)
parameters = np.array(df)
params = parameters.ravel()
calculator = change_calc(supercell_basal,calculator,params)
supercell_pyrm_2 = read("/home/f/PhD/ace_pot/supercells/1_1_-2_2.cfg",format='cfg')
supercell_pyrm_2 = supercell_pyrm_2 * (1,1,2)
v1 = supercell_pyrm_2.cell[0]
v2 = supercell_pyrm_2.cell[1]
slip_vec = np.array([0,5.49839340,0])
create_gamma_line(supercell_pyrm_2,-slip_vec,"gamma_line_pyrm2.xyz",calculator,3)

In [ ]:
#This computes the average max force and average RMS force per gamma line
images = read("my_and_connor.xyz", index=":")
num_configs = len(images)
print("Number of Structures: ", num_configs)

qm_forces = [atoms.arrays["QM_forces"] for atoms in images]



max_force_avgs = []
rms_force_avgs = []


gamma_line_forces = qm_forces[7146:7155]  # list of arrays

max_forces = []
rms_forces = []

for f in gamma_line_forces:  # f is (N_atoms, 3)
    z_forces = f[:, 2]  # take only z-component
    max_forces.append(np.max(np.abs(z_forces)))   # max |Fz| in structure
    rms_forces.append(np.sqrt(np.mean(z_forces**2)))  # RMS of Fz in structure

max_force_avg = np.mean(max_forces)
rms_force_avg = np.mean(rms_forces)

max_force_avgs.append(max_force_avg)
rms_force_avgs.append(rms_force_avg)

print(f"Pointer {i}: Avg Max Fz = {max_force_avg:.6f}, Avg RMS Fz = {rms_force_avg:.6f}")
